In [4]:
# https://github.com/dqwei-lab/SPVec/blob/master/source%20codes/feature_representation.py
import numpy as np
import pandas as pd
np.set_printoptions(threshold=123456789)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import math, random
from collections import Counter
from itertools import product
from copy import deepcopy

# from rdkit import Chem
# from rdkit.Chem import AllChem
import os, re

import gensim 
import jieba
from gensim.models import Word2Vec

In [2]:
pwd = '/home/chujunyi/2_Program/2_output_file/1_construct_dataset/3_drop_repeat_smiles_from2folder/'
drug_id_smiles_with_target = pd.read_csv(pwd + 'v2_nr_updated_drug_smiles_ids_drop_repeated.csv')
drug_id_smiles_with_target.shape

(886, 3)

# generate word2vec

In [60]:
def word2vec(data, dims = 100, window_size = 6, negative_size = 15):

    texts = [[word for word in re.findall(r'.{2}',document)] for document in list(data)]
    model = Word2Vec(texts, size = dims, window = window_size, negative = negative_size,
                     min_count = 1, sg = 1, sample = 0.001, hs = 1, workers = 4)
    vectors = pd.DataFrame([model[word] for word in (model.wv.vocab)])
    vectors['Word'] = list(model.wv.vocab)
    
    Index = list(range(len(data)))
    word_vec = pd.DataFrame()
    dictionary_idx, dictionary_text = [], []
    for i in range(len(texts)):
        idx_word, text_word = [], []  
        for w in range(len(texts[i])):
            idx_word.append(Index[i])  # word segmentation
            text_word.append(texts[i][w]) # word vectors generation
        dictionary_idx.extend(idx_word)
        dictionary_text.extend(text_word)
    word_vec['Id'] = dictionary_idx
    word_vec['Word'] = dictionary_text
    word_vec = word_vec.merge(vectors, on = 'Word', how = 'left')
    word_vec.columns = ['Id'] + ['Word'] + ["vec_{0}".format(i) for i in range(0,dims)]
    
    print('word_vec.shape = {}'.format(word_vec.shape))
    return word_vec

In [26]:
def feature_embeddings(word_vec, dims = 100):
    
    try:
        del word_vec['Word']
    except:
        None
        
    col_name = ["vec_{0}".format(i) for i in range(0,dims)]
    feature_embeddings = pd.DataFrame(word_vec.groupby(['Id'])[col_name].agg('mean')).reset_index()
    feature_embeddings.columns=["Index"]+["mean_ci_{0}".format(i) for i in range(0,dims)]
    
    print('feature shape = {}'.format(feature_embeddings.shape))
    return feature_embeddings

In [55]:
def generate_w2v(target = 'GPCR'):
    pwd = '/home/chujunyi/2_Program/2_output_file/1_construct_dataset/3_drop_repeat_smiles_from2folder/'
    drug_id_smiles_with_target = pd.read_csv(pwd + 'v2_' + target + '_updated_drug_smiles_ids_drop_repeated.csv')
    print(drug_id_smiles_with_target.shape)
    
    drug_id_smiles = deepcopy(drug_id_smiles_with_target.iloc[:, :2])
    drug_id_smiles = drug_id_smiles.drop_duplicates().reset_index(drop = True)
    print(drug_id_smiles.shape)
    
    drug_vec = word2vec(drug_id_smiles['smiles'], dims = 100, window_size = 6, negative_size = 15)
    drug_embeddings = feature_embeddings(drug_vec, dims = 100)
    drug_embeddings['Index'] = drug_id_smiles['drug_id']
    drug_embeddings.rename({'Index':'drug_id'}, axis='columns', inplace = True)
    return drug_id_smiles_with_target, drug_embeddings

In [3]:
drug_id_smiles = deepcopy(drug_id_smiles_with_target.iloc[:, :2])
drug_id_smiles = drug_id_smiles.drop_duplicates().reset_index(drop = True)
drug_id_smiles.shape

(541, 2)

In [9]:
drug_vec = word2vec(drug_id_smiles['smiles'], dims = 100, window_size = 6, negative_size = 15)
drug_embeddings = feature_embeddings(drug_vec, dims = 100)
# drug_embeddings['smiles'] = drug_id_smiles['smiles']
drug_embeddings['Index'] = drug_id_smiles['drug_id']
drug_embeddings.rename({'Index':'drug_id'}, axis='columns', inplace = True)
drug_embeddings.head()

word_vec.shape = (9154, 102)
feature shape = (541, 101)


/home/chujunyi/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


,drug_id,mean_ci_0,mean_ci_1,mean_ci_2,mean_ci_3,mean_ci_4,mean_ci_5,mean_ci_6,mean_ci_7,mean_ci_8,mean_ci_9,mean_ci_10,mean_ci_11,mean_ci_12,mean_ci_13,mean_ci_14,mean_ci_15,mean_ci_16,mean_ci_17,mean_ci_18,mean_ci_19,mean_ci_20,mean_ci_21,mean_ci_22,mean_ci_23,mean_ci_24,mean_ci_25,mean_ci_26,mean_ci_27,mean_ci_28,mean_ci_29,mean_ci_30,mean_ci_31,mean_ci_32,mean_ci_33,mean_ci_34,mean_ci_35,mean_ci_36,mean_ci_37,mean_ci_38,mean_ci_39,mean_ci_40,mean_ci_41,mean_ci_42,mean_ci_43,mean_ci_44,mean_ci_45,mean_ci_46,mean_ci_47,mean_ci_48,mean_ci_49,mean_ci_50,mean_ci_51,mean_ci_52,mean_ci_53,mean_ci_54,mean_ci_55,mean_ci_56,mean_ci_57,mean_ci_58,mean_ci_59,mean_ci_60,mean_ci_61,mean_ci_62,mean_ci_63,mean_ci_64,mean_ci_65,mean_ci_66,mean_ci_67,mean_ci_68,mean_ci_69,mean_ci_70,mean_ci_71,mean_ci_72,mean_ci_73,mean_ci_74,mean_ci_75,mean_ci_76,mean_ci_77,mean_ci_78,mean_ci_79,mean_ci_80,mean_ci_81,mean_ci_82,mean_ci_83,mean_ci_84,mean_ci_85,mean_ci_86,mean_ci_87,mean_ci_88,mean_ci_89,mean_ci_90,mean_ci_91,mean_ci_92,mean_ci_93,mean_ci_94,mean_ci_95,mean_ci_96,mean_ci_97,mean_ci_98,mean_ci_99
0,D07800,-0.316674,-0.124278,0.220964,0.053072,0.182712,-0.145850,0.119570,-0.134152,-0.422940,-0.027600,-0.121020,-0.245322,-0.121516,-0.058690,0.030506,-0.381433,-0.205843,0.234212,-0.279494,-0.238186,0.324299,-0.107014,0.087931,0.075978,0.089218,0.060136,-0.358474,-0.057664,-0.298693,0.195412,0.311334,-0.302271,0.037459,-0.221841,-0.316457,-0.343625,0.268875,-0.026156,-0.134938,0.187909,-0.128321,-0.071684,0.303425,-0.121794,-0.144675,-0.129802,-0.211669,-0.204119,0.011414,-0.037331,-0.182138,-0.220325,0.197819,-0.278776,-0.025841,0.091471,-0.058127,-0.112421,-0.312531,-0.520756,0.044392,-0.303947,-0.078621,0.233118,0.164352,0.100206,0.311157,0.158625,0.198844,0.298944,-0.006659,-0.035259,0.109387,-0.157446,0.052360,-0.497446,0.098909,0.221981,-0.184544,0.016100,0.040373,-0.273880,-0.279095,-0.065537,0.268517,-0.338021,-0.143647,-0.023256,0.289473,0.075296,0.602077,-0.060110,0.115247,0.138245,0.203314,-0.216554,-0.154474,0.115957,0.205189,-0.268239
1,D00088,-0.289568,-0.182173,0.176405,-0.018498,0.215831,-0.169039,0.088991,-0.116300,-0.407232,-0.081493,-0.031863,-0.143433,-0.036772,-0.108565,0.001004,-0.343310,-0.219495,0.276757,-0.226049,-0.212679,0.384644,-0.183344,0.083682,0.053394,0.075792,-0.024942,-0.294537,-0.094600,-0.304106,0.189305,0.333712,-0.255536,0.020726,-0.107148,-0.361893,-0.301957,0.344672,0.032224,-0.104065,0.212644,-0.175061,-0.125686,0.280703,-0.051261,-0.113503,-0.123978,-0.128323,-0.084597,-0.086942,-0.091651,-0.153126,-0.331872,0.156720,-0.224946,-0.073429,0.035483,0.056407,-0.045051,-0.338595,-0.521039,0.011008,-0.328095,-0.111563,0.290926,0.152642,0.056018,0.298543,0.108228,0.172263,0.219487,-0.007250,0.063487,0.112545,-0.127446,0.035846,-0.456502,-0.013711,0.218959,-0.212477,-0.048851,0.043711,-0.344541,-0.260903,0.003666,0.181929,-0.280596,-0.246939,-0.030645,0.358550,0.104873,0.524889,-0.028394,0.111096,0.069662,0.132881,-0.165820,-0.158508,0.066721,0.287047,-0.197370
2,D06876,-0.299233,-0.116686,0.194946,0.042018,0.188154,-0.153934,0.120570,-0.090432,-0.392025,-0.045268,-0.092168,-0.235401,-0.074203,-0.045895,0.027543,-0.362646,-0.190453,0.199616,-0.255676,-0.242160,0.332143,-0.106662,0.091268,0.065442,0.091247,0.070407,-0.340311,-0.066154,-0.273674,0.177710,0.291095,-0.262123,0.033247,-0.199583,-0.300841,-0.340742,0.287779,0.016920,-0.149042,0.184249,-0.112868,-0.100269,0.292618,-0.080811,-0.113020,-0.128980,-0.199164,-0.184016,-0.013665,-0.040748,-0.169916,-0.235491,0.167222,-0.290561,-0.021856,0.050903,-0.048273,-0.121201,-0.311720,-0.488037,0.033364,-0.312888,-0.100379,0.221265,0.166955,0.095164,0.281306,0.133510,0.196288,0.279501,0.007900,0.006730,0.110753,-0.135684,0.027562,-0.473094,0.073088,0.220101,-0.197298,-0.016918,0.052663,-0.256682,-0.256367,-0.033345,0.241978,-0.301422,-0.157727,-0.025644,0.289038,0.077982,0.556795,-0.062375,0.117914,0.095961,0.166085,-0.199364,-0.169350,0.096613,0.223955,-0.214036
3,D

# generate XY from word2vec feature

In [13]:
def generate_dti_data_with_drug_feature(dti_data, drug_feature):

    label_num = len(set(dti_data['hsa_id']))
    
    data_target_label = pd.concat([dti_data, pd.get_dummies(dti_data['hsa_id'])], axis = 1).groupby(by = 'drug_id').sum().reset_index()
    data_target_label = pd.merge(drug_feature, data_target_label, on = 'drug_id').sample(frac = 1.0, random_state = 1231).reset_index(drop = True)
    print(data_target_label.shape)
    x_target_label = data_target_label.iloc[:, 1:-label_num] # 1 columns: drug_id
    y_target_label = data_target_label.iloc[:, -label_num:]
    assert x_target_label.shape[1] == drug_feature.shape[1] - 1 # 1 columns: drug_id
    assert y_target_label.shape[1] == label_num
    
    label_names = np.array(y_target_label.columns)
    drug_names = np.array(data_target_label['drug_id'])
    
    x_t_np = np.array(x_target_label)
    y_t_np = np.array(y_target_label)
    print('X.shape = {}, Y.shape = {}'.format(x_t_np.shape, y_t_np.shape))
    
    return drug_names, label_names, x_t_np, y_t_np

In [20]:
label_names, x_t_np_word2vec, y_t_np_word2vec = generate_dti_data_with_drug_feature(drug_id_smiles_with_target, drug_embeddings)

(541, 134)
X.shape = (541, 100), Y.shape = (541, 33)


In [22]:
pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
np.savez_compressed(pwd + 'NR_U_T_xy_np_word2vec.npz', label_names = label_names, x_t_np = x_t_np_word2vec, y_t_np = y_t_np_word2vec)

In [21]:
gpcr_drug_id_smiles_with_target, gpcr_drug_embeddings = generate_w2v(target = 'GPCR')
gpcr_drug_names, gpcr_label_names, gpcr_x_t_np_word2vec, gpcr_y_t_np_word2vec = generate_dti_data_with_drug_feature(gpcr_drug_id_smiles_with_target, gpcr_drug_embeddings)
pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
np.savez_compressed(pwd + 'GPCR_U_T_xy_np_word2vec.npz', 
                    drugs_names = gpcr_drug_names,
                    label_names = gpcr_label_names, 
                    x_t_np = gpcr_x_t_np_word2vec, 
                    y_t_np = gpcr_y_t_np_word2vec,
                    norm_idx = list(range(gpcr_x_t_np_word2vec.shape[1])))

(5383, 3)
(1680, 2)


/home/chujunyi/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


word_vec.shape = (29460, 102)
feature shape = (1680, 101)
(1680, 257)
X.shape = (1680, 100), Y.shape = (1680, 156)


In [22]:
ic_drug_id_smiles_with_target, ic_drug_embeddings = generate_w2v(target = 'IC')
ic_drug_names, ic_label_names, ic_x_t_np_word2vec, ic_y_t_np_word2vec = generate_dti_data_with_drug_feature(ic_drug_id_smiles_with_target, ic_drug_embeddings)
pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
np.savez_compressed(pwd + 'IC_U_T_xy_np_word2vec.npz', 
                    drugs_names = ic_drug_names,
                    label_names = ic_label_names, 
                    x_t_np = ic_x_t_np_word2vec, 
                    y_t_np = ic_y_t_np_word2vec,
                    norm_idx = list(range(ic_x_t_np_word2vec.shape[1])))

(6385, 3)
(765, 2)


/home/chujunyi/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


word_vec.shape = (10679, 102)
feature shape = (765, 101)
(765, 339)
X.shape = (765, 100), Y.shape = (765, 238)


In [23]:
e_drug_id_smiles_with_target, e_drug_embeddings = generate_w2v(target = 'E')
e_drug_names, e_label_names, e_x_t_np_word2vec, e_y_t_np_word2vec = generate_dti_data_with_drug_feature(e_drug_id_smiles_with_target, e_drug_embeddings)
pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
np.savez_compressed(pwd + 'E_U_T_xy_np_word2vec.npz', 
                    drugs_names = e_drug_names,
                    label_names = e_label_names, 
                    x_t_np = e_x_t_np_word2vec, 
                    y_t_np = e_y_t_np_word2vec,
                    norm_idx = list(range(e_x_t_np_word2vec.shape[1])))

(7371, 3)
(1777, 2)


/home/chujunyi/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


word_vec.shape = (29764, 102)
feature shape = (1776, 101)
(1776, 1512)
X.shape = (1776, 100), Y.shape = (1776, 1411)


In [62]:
e_drug_id_smiles_with_target, e_drug_embeddings = generate_w2v(target = 'E')
e_drug_names, e_label_names, e_x_t_np_word2vec, e_y_t_np_word2vec = generate_dti_data_with_drug_feature(e_drug_id_smiles_with_target, e_drug_embeddings)
pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
np.savez_compressed(pwd + 'v2_E_U_T_xy_np_word2vec.npz', 
                    drugs_names = e_drug_names,
                    label_names = e_label_names, 
                    x_t_np = e_x_t_np_word2vec, 
                    y_t_np = e_y_t_np_word2vec,
                    norm_idx = list(range(e_x_t_np_word2vec.shape[1])))

(7371, 3)
(1777, 2)


/home/chujunyi/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


word_vec.shape = (45127, 102)
feature shape = (1777, 101)
(1777, 1512)
X.shape = (1777, 100), Y.shape = (1777, 1411)
